In [1]:
import sqlite3

In [2]:
def create_empty_database(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except sqlite3.Error as e:
        print(e)
    finally:
        if conn:
            conn.close()

def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except sqlite3.Error as e:
        print(e)

    return conn

def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except sqlite3.Error as e:
        print(e)


def add_data_entry(conn, cols, entry):
    """
    Create a new project into the projects table
    :param conn:
    :param cols:
    :param entry:
    :return: project id
    """
    cur = conn.cursor()
    cur.execute(cols, entry)
    return cur.lastrowid

In [3]:
def create_primary_table(conn):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    create_table_sql = """
    CREATE TABLE IF NOT EXISTS datasets (
    id integer PRIMARY KEY,
    species text NOT NULL,
    organ_or_cell_line text NOT NULL,
    reference_genome test NOT NULL,
    assay text NOT NULL,
    whether_processed bit,
    format text,
    linked_table text,
    path text,
    ref text,
    additional_information text
    );
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except sqlite3.Error as e:
        print(e)


def add_primary_data_entry(conn, entry):
    """
    Create a new project into the projects table
    :param conn:
    :param project:
    :return: project id
    """
    sql = '''
    INSERT INTO datasets(species, organ_or_cell_line, reference_genome, assay, whether_processed, format,
    linked_table, path, ref, additional_information)
    VALUES(?,?,?,?,?,?,?,?,?,?)
    '''
    cur = conn.cursor()
    cur.execute(sql, entry)
    return cur.lastrowid

Create an empty database.

Build a primary table for storing all datasets we have.

In [4]:
create_empty_database('Liu_lab_datasets.db')
conn = create_connection('Liu_lab_datasets.db')

create_primary_table(conn)

2.6.0


In [5]:
d1 = ('mouse', 'embryonic stem cell', 'mm10', 0, 'scHiC', 'text', 'scHiC',
      '/nfs/turbo/umms-drjieliu/proj/4dn/data/scHiC/Nagano/raw',
      'https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE94489', 'NULL')
d2 = ('mouse', 'embryonic stem cell', 'mm9', 'scHiC', 1, 'text', 'scHiC',
      '/nfs/turbo/umms-drjieliu/proj/4dn/data/scHiC/Nagano/processed',
      'https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE94489', 'NULL')

id1 = add_primary_data_entry(conn, d1)
id2 = add_primary_data_entry(conn, d2)

Build a secondary table for single-cell Hi-C

In [6]:
create_table_sql = """
    CREATE TABLE IF NOT EXISTS scHiC (
    id integer PRIMARY KEY,
    species text NOT NULL,
    organ_or_cell_line text NOT NULL,
    reference_genome test NOT NULL,
    cell_id test NOT NULL,
    whether_processed text,
    format text,
    dataset_id integer,
    path text,
    ref text,
    additional_information text
    );
    """

create_table(conn, create_table_sql)

cols = '''
    INSERT INTO scHiC(species, organ_or_cell_line, reference_genome, cell_id, whether_processed, format,
    dataset_id, path, ref, additional_information)
    VALUES(?,?,?,?,?,?,?,?,?,?)
    '''
p1 = ('mouse', 'embryonic stem cell', 'mm9', '1CDX.1', 1, 'text', id2,
      '/nfs/turbo/umms-drjieliu/proj/4dn/data/scHiC/Nagano/processed/500kb/schic_hyb_1CDX1_adj_files/1CDX1.1/new_adj',
      'https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE94489', 'resolution:500kb')
p2 = ('mouse', 'embryonic stem cell', 'mm9', '1CDX.3', 1, 'text', id2,
      '/nfs/turbo/umms-drjieliu/proj/4dn/data/scHiC/Nagano/processed/500kb/schic_hyb_1CDX1_adj_files/1CDX1.3/new_adj',
      'https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE94489', 'resolution:500kb')

a = add_data_entry(conn, cols, p1)
b = add_data_entry(conn, cols, p2)

In [7]:
conn.commit()